In [5]:
!pip install selenium beautifulsoup4 pandas


[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [7]:
# Function to initialize WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument('--headless')  # Uncomment this line to run Chrome in headless mode
    driver = webdriver.Chrome(options=options)
    return driver


In [8]:
# Initialize the WebDriver (e.g., for Chrome)
driver = initialize_driver()

# Define the URL
url = 'https://www.cars24.com/buy-used-car?f=make%3A%3D%3Ahyundai&sort=bestmatch&serveWarrantyCount=true&listingSource=Homepage_Filters&storeCityId=2378'

# Open the URL
driver.get(url)


In [9]:
# Wait for the page to load and display the cars
wait = WebDriverWait(driver, 10)
cars_loaded = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_2YB7p')))


In [10]:
# Scroll and load all cars
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll to the bottom of the page to load more cars
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)  # Give time for the next batch of cars to load
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Break the loop if no more cars are loaded
    last_height = new_height


In [11]:
# Once all cars are loaded, get the page source and parse it with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


In [12]:
# Initialize lists to store the data
names = []
kilometers_driven = []
fuel_type = []
transmission = []
price = []


In [13]:
# Scrape the data
cars = soup.find_all('div', class_='_2YB7p')
for car in cars:
    names.append(car.find('h3').get_text())
    km = car.find('ul', {'class': '_3J2G-'}).find_all('li')[0].get_text()
    fuel = car.find('ul', {'class': '_3J2G-'}).find_all('li')[2].get_text()
    trans = car.find('ul', {'class': '_3J2G-'}).find_all('li')[-1].get_text()
    pr = car.find('strong', {'class': '_3RL-I'}).get_text()

    kilometers_driven.append(km)
    fuel_type.append(fuel)
    transmission.append(trans)
    price.append(pr)


In [14]:
# Create a dictionary with the data
data = {
    'Name': names,
    'Kilometers Driven': kilometers_driven,
    'Fuel Type': fuel_type,
    'Transmission': transmission,
    'Price': price
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('cars_data.csv', index=False)

print("Data has been saved to cars_data.csv")


Data has been saved to cars_data.csv


In [15]:
print(names)

['2014 Hyundai Xcent S 1.2', '2015 Hyundai Elite i20 SPORTZ 1.2', '2016 Hyundai Elite i20 ASTA 1.2 (O)', '2015 Hyundai Elite i20 MAGNA 1.2', '2016 Hyundai Elite i20 SPORTZ 1.2', '2018 Hyundai Elite i20 ASTA 1.2 (O)', '2016 Hyundai Creta SX PLUS AT 1.6 PETROL', '2015 Hyundai Grand i10 ASTA AT 1.2 KAPPA VTVT', '2015 Hyundai Elite i20 SPORTZ 1.2 (O)', '2018 Hyundai Grand i10 MAGNA 1.2 KAPPA VTVT', '2016 Hyundai Grand i10 SPORTZ 1.2 KAPPA VTVT', '2017 Hyundai Elite i20 SPORTZ 1.2', '2021 Hyundai VENUE SX 1.0 TURBO IMT', '2021 Hyundai VENUE SX PLUS 1.0 TURBO DCT', '2016 Hyundai Elite i20 MAGNA 1.2', '2019 Hyundai GRAND I10 NIOS SPORTZ 1.2 KAPPA VTVT DUAL TONE', '2014 Hyundai Eon MAGNA +', '2019 Hyundai VENUE SX 1.0 (O) TURBO', '2021 Hyundai VENUE S 1.0 TURBO IMT', '2019 Hyundai VENUE SX 1.0 TURBO', '2018 Hyundai NEW SANTRO MAGNA CNG', '2014 Hyundai Elite i20 SPORTZ 1.2', '2017 Hyundai Elite i20 SPORTZ 1.2', '2016 Hyundai Elite i20 ASTA 1.2', '2014 Hyundai Xcent BASE 1.2', '2016 Hyundai Gran

In [16]:
len(names)

154